# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([24, 77,  6, 80, 34, 47, 42, 63, 33, 25, 25, 25, 67,  6, 80, 80, 50,
       63, 56,  6, 38, 59, 21, 59, 47, 69, 63,  6, 42, 47, 63,  6, 21, 21,
       63,  6, 21, 59,  4, 47, 35, 63, 47, 28, 47, 42, 50, 63, 16, 22, 77,
        6, 80, 80, 50, 63, 56,  6, 38, 59, 21, 50, 63, 59, 69, 63, 16, 22,
       77,  6, 80, 80, 50, 63, 59, 22, 63, 59, 34, 69, 63, 37, 43, 22, 25,
       43,  6, 50, 52, 25, 25, 82, 28, 47, 42, 50, 34, 77, 59, 22])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [99]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = int((len(arr)-batch_size)/(batch_size*seq_length))
    #print(f'n_batches {n_batches}')
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:batch_size*seq_length*n_batches+batch_size]
    
    ## TODO: Reshape into batch_size rows
    arr = np.reshape(arr, (batch_size, -1))
    #print(f'arr.shape {arr.shape}')
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1]-1, seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = arr[:, n+1:n+seq_length+1]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [100]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [101]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[24 77  6 80 34 47 42 63 33 25]
 [37 22 63 34 77  6 34 63  6 34]
 [71 63 37 42 63  6 63 56 37 47]
 [77 47 63 26 77 59 47 56 63 34]
 [63 77 47 42 63 34 47  6 42 75]
 [37 22 63  6 22 71 63  6 22  6]
 [77  6 71 63 69  6 59 71 63 34]
 [50 52 63 15 74 16 34 63 27 34]]

y
 [[77  6 80 34 47 42 63 33 25 25]
 [22 63 34 77  6 34 63  6 34 34]
 [63 37 42 63  6 63 56 37 47 51]
 [47 63 26 77 59 47 56 63 34 77]
 [77 47 42 63 34 47  6 42 75 69]
 [22 63  6 22 71 63  6 22  6 21]
 [ 6 71 63 69  6 59 71 63 34 77]
 [52 63 15 74 16 34 63 27 34 77]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [102]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [103]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), self.n_hidden, self.n_layers, 
                            dropout=self.drop_prob, batch_first=True)
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out, hidden = self.lstm(x, hidden)
        #print(f'out.shape {out.shape}')
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [104]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            #print(f'h {h}')

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.reshape(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.reshape(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [105]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=256
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [106]:
batch_size = 8
seq_length = 50
n_epochs =  10# start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/10... Step: 10... Loss: 3.2115... Val Loss: 3.2353
Epoch: 1/10... Step: 20... Loss: 3.0746... Val Loss: 3.1561
Epoch: 1/10... Step: 30... Loss: 3.1672... Val Loss: 3.1510
Epoch: 1/10... Step: 40... Loss: 3.0845... Val Loss: 3.1392
Epoch: 1/10... Step: 50... Loss: 3.0024... Val Loss: 3.1352
Epoch: 1/10... Step: 60... Loss: 3.0387... Val Loss: 3.1469
Epoch: 1/10... Step: 70... Loss: 3.0026... Val Loss: 3.1392
Epoch: 1/10... Step: 80... Loss: 3.0110... Val Loss: 3.1388
Epoch: 1/10... Step: 90... Loss: 3.0820... Val Loss: 3.1413
Epoch: 1/10... Step: 100... Loss: 3.0795... Val Loss: 3.1372
Epoch: 1/10... Step: 110... Loss: 3.1350... Val Loss: 3.1363
Epoch: 1/10... Step: 120... Loss: 3.0707... Val Loss: 3.1401
Epoch: 1/10... Step: 130... Loss: 3.0395... Val Loss: 3.1334
Epoch: 1/10... Step: 140... Loss: 3.0373... Val Loss: 3.1306
Epoch: 1/10... Step: 150... Loss: 3.2028... Val Loss: 3.1419
Epoch: 1/10... Step: 160... Loss: 3.0541... Val Loss: 3.1357
Epoch: 1/10... Step: 170... Loss:

Epoch: 1/10... Step: 1350... Loss: 2.1219... Val Loss: 2.1502
Epoch: 1/10... Step: 1360... Loss: 2.1276... Val Loss: 2.1416
Epoch: 1/10... Step: 1370... Loss: 2.0250... Val Loss: 2.1403
Epoch: 1/10... Step: 1380... Loss: 2.0246... Val Loss: 2.1449
Epoch: 1/10... Step: 1390... Loss: 2.0833... Val Loss: 2.1466
Epoch: 1/10... Step: 1400... Loss: 1.9888... Val Loss: 2.1400
Epoch: 1/10... Step: 1410... Loss: 1.9485... Val Loss: 2.1331
Epoch: 1/10... Step: 1420... Loss: 2.0415... Val Loss: 2.1350
Epoch: 1/10... Step: 1430... Loss: 2.0406... Val Loss: 2.1271
Epoch: 1/10... Step: 1440... Loss: 2.1278... Val Loss: 2.1309
Epoch: 1/10... Step: 1450... Loss: 2.1336... Val Loss: 2.1296
Epoch: 1/10... Step: 1460... Loss: 2.0674... Val Loss: 2.1207
Epoch: 1/10... Step: 1470... Loss: 2.0174... Val Loss: 2.1189
Epoch: 1/10... Step: 1480... Loss: 2.0692... Val Loss: 2.1201
Epoch: 1/10... Step: 1490... Loss: 2.1422... Val Loss: 2.1113
Epoch: 1/10... Step: 1500... Loss: 2.0406... Val Loss: 2.1088
Epoch: 1

Epoch: 1/10... Step: 2680... Loss: 1.8295... Val Loss: 1.8827
Epoch: 1/10... Step: 2690... Loss: 1.8175... Val Loss: 1.8769
Epoch: 1/10... Step: 2700... Loss: 1.8826... Val Loss: 1.8797
Epoch: 1/10... Step: 2710... Loss: 1.8503... Val Loss: 1.8824
Epoch: 1/10... Step: 2720... Loss: 1.7406... Val Loss: 1.8756
Epoch: 1/10... Step: 2730... Loss: 1.7540... Val Loss: 1.8747
Epoch: 1/10... Step: 2740... Loss: 1.7194... Val Loss: 1.8714
Epoch: 1/10... Step: 2750... Loss: 1.8807... Val Loss: 1.8645
Epoch: 1/10... Step: 2760... Loss: 1.7798... Val Loss: 1.8629
Epoch: 1/10... Step: 2770... Loss: 1.9031... Val Loss: 1.8592
Epoch: 1/10... Step: 2780... Loss: 1.8553... Val Loss: 1.8631
Epoch: 1/10... Step: 2790... Loss: 1.8060... Val Loss: 1.8590
Epoch: 1/10... Step: 2800... Loss: 1.9003... Val Loss: 1.8704
Epoch: 1/10... Step: 2810... Loss: 1.8152... Val Loss: 1.8661
Epoch: 1/10... Step: 2820... Loss: 1.8597... Val Loss: 1.8560
Epoch: 1/10... Step: 2830... Loss: 1.8880... Val Loss: 1.8543
Epoch: 1

Epoch: 1/10... Step: 4010... Loss: 1.5786... Val Loss: 1.7321
Epoch: 1/10... Step: 4020... Loss: 1.4967... Val Loss: 1.7355
Epoch: 1/10... Step: 4030... Loss: 1.7787... Val Loss: 1.7311
Epoch: 1/10... Step: 4040... Loss: 1.6537... Val Loss: 1.7249
Epoch: 1/10... Step: 4050... Loss: 1.6332... Val Loss: 1.7290
Epoch: 1/10... Step: 4060... Loss: 1.5278... Val Loss: 1.7281
Epoch: 1/10... Step: 4070... Loss: 1.6049... Val Loss: 1.7203
Epoch: 1/10... Step: 4080... Loss: 1.6071... Val Loss: 1.7230
Epoch: 1/10... Step: 4090... Loss: 1.4837... Val Loss: 1.7273
Epoch: 1/10... Step: 4100... Loss: 1.4119... Val Loss: 1.7244
Epoch: 1/10... Step: 4110... Loss: 1.4836... Val Loss: 1.7377
Epoch: 1/10... Step: 4120... Loss: 1.6410... Val Loss: 1.7288
Epoch: 1/10... Step: 4130... Loss: 1.5633... Val Loss: 1.7235
Epoch: 1/10... Step: 4140... Loss: 1.8053... Val Loss: 1.7246
Epoch: 1/10... Step: 4150... Loss: 1.6308... Val Loss: 1.7211
Epoch: 1/10... Step: 4160... Loss: 1.5843... Val Loss: 1.7243
Epoch: 1

Epoch: 2/10... Step: 5340... Loss: 1.6054... Val Loss: 1.6395
Epoch: 2/10... Step: 5350... Loss: 1.4148... Val Loss: 1.6413
Epoch: 2/10... Step: 5360... Loss: 1.6589... Val Loss: 1.6420
Epoch: 2/10... Step: 5370... Loss: 1.6238... Val Loss: 1.6382
Epoch: 2/10... Step: 5380... Loss: 1.5602... Val Loss: 1.6377
Epoch: 2/10... Step: 5390... Loss: 1.4491... Val Loss: 1.6424
Epoch: 2/10... Step: 5400... Loss: 1.5561... Val Loss: 1.6344
Epoch: 2/10... Step: 5410... Loss: 1.5347... Val Loss: 1.6353
Epoch: 2/10... Step: 5420... Loss: 1.4264... Val Loss: 1.6351
Epoch: 2/10... Step: 5430... Loss: 1.4275... Val Loss: 1.6313
Epoch: 2/10... Step: 5440... Loss: 1.3634... Val Loss: 1.6373
Epoch: 2/10... Step: 5450... Loss: 1.5465... Val Loss: 1.6342
Epoch: 2/10... Step: 5460... Loss: 1.5903... Val Loss: 1.6301
Epoch: 2/10... Step: 5470... Loss: 1.4081... Val Loss: 1.6330
Epoch: 2/10... Step: 5480... Loss: 1.3956... Val Loss: 1.6368
Epoch: 2/10... Step: 5490... Loss: 1.5243... Val Loss: 1.6343
Epoch: 2

...

Epoch: 9/10... Step: 39470... Loss: 1.2834... Val Loss: 1.3674
Epoch: 9/10... Step: 39480... Loss: 1.1818... Val Loss: 1.3675
Epoch: 9/10... Step: 39490... Loss: 1.1960... Val Loss: 1.3727
Epoch: 9/10... Step: 39500... Loss: 1.1317... Val Loss: 1.3706
Epoch: 9/10... Step: 39510... Loss: 1.1297... Val Loss: 1.3699
Epoch: 9/10... Step: 39520... Loss: 1.3505... Val Loss: 1.3704
Epoch: 9/10... Step: 39530... Loss: 1.1919... Val Loss: 1.3669
Epoch: 9/10... Step: 39540... Loss: 1.1398... Val Loss: 1.3683
Epoch: 9/10... Step: 39550... Loss: 1.2098... Val Loss: 1.3675
Epoch: 9/10... Step: 39560... Loss: 1.1794... Val Loss: 1.3679
Epoch: 9/10... Step: 39570... Loss: 1.2848... Val Loss: 1.3700
Epoch: 9/10... Step: 39580... Loss: 1.3103... Val Loss: 1.3705
Epoch: 9/10... Step: 39590... Loss: 1.2908... Val Loss: 1.3716
Epoch: 9/10... Step: 39600... Loss: 1.2444... Val Loss: 1.3723
Epoch: 9/10... Step: 39610... Loss: 1.1594... Val Loss: 1.3720
Epoch: 9/10... Step: 39620... Loss: 1.2178... Val Loss:

Epoch: 10/10... Step: 40770... Loss: 1.0892... Val Loss: 1.3725
Epoch: 10/10... Step: 40780... Loss: 1.1951... Val Loss: 1.3739
Epoch: 10/10... Step: 40790... Loss: 1.2928... Val Loss: 1.3730
Epoch: 10/10... Step: 40800... Loss: 0.9271... Val Loss: 1.3732
Epoch: 10/10... Step: 40810... Loss: 1.1759... Val Loss: 1.3696
Epoch: 10/10... Step: 40820... Loss: 1.2934... Val Loss: 1.3706
Epoch: 10/10... Step: 40830... Loss: 1.0869... Val Loss: 1.3730
Epoch: 10/10... Step: 40840... Loss: 1.0790... Val Loss: 1.3730
Epoch: 10/10... Step: 40850... Loss: 1.0768... Val Loss: 1.3723
Epoch: 10/10... Step: 40860... Loss: 1.0648... Val Loss: 1.3732
Epoch: 10/10... Step: 40870... Loss: 1.3251... Val Loss: 1.3736
Epoch: 10/10... Step: 40880... Loss: 1.2823... Val Loss: 1.3715
Epoch: 10/10... Step: 40890... Loss: 1.1669... Val Loss: 1.3709
Epoch: 10/10... Step: 40900... Loss: 1.1398... Val Loss: 1.3674
Epoch: 10/10... Step: 40910... Loss: 1.2314... Val Loss: 1.3698
Epoch: 10/10... Step: 40920... Loss: 1.2

Epoch: 10/10... Step: 42060... Loss: 1.1826... Val Loss: 1.3749
Epoch: 10/10... Step: 42070... Loss: 1.1882... Val Loss: 1.3728
Epoch: 10/10... Step: 42080... Loss: 1.3531... Val Loss: 1.3717
Epoch: 10/10... Step: 42090... Loss: 1.1269... Val Loss: 1.3712
Epoch: 10/10... Step: 42100... Loss: 1.2533... Val Loss: 1.3720
Epoch: 10/10... Step: 42110... Loss: 1.0609... Val Loss: 1.3730
Epoch: 10/10... Step: 42120... Loss: 1.1949... Val Loss: 1.3733
Epoch: 10/10... Step: 42130... Loss: 1.2685... Val Loss: 1.3708
Epoch: 10/10... Step: 42140... Loss: 1.1967... Val Loss: 1.3705
Epoch: 10/10... Step: 42150... Loss: 1.1670... Val Loss: 1.3714
Epoch: 10/10... Step: 42160... Loss: 1.1590... Val Loss: 1.3717
Epoch: 10/10... Step: 42170... Loss: 1.1619... Val Loss: 1.3704
Epoch: 10/10... Step: 42180... Loss: 1.2233... Val Loss: 1.3731
Epoch: 10/10... Step: 42190... Loss: 1.2688... Val Loss: 1.3728
Epoch: 10/10... Step: 42200... Loss: 1.1336... Val Loss: 1.3755
Epoch: 10/10... Step: 42210... Loss: 1.0

Epoch: 10/10... Step: 43350... Loss: 1.0014... Val Loss: 1.3674
Epoch: 10/10... Step: 43360... Loss: 1.1091... Val Loss: 1.3678
Epoch: 10/10... Step: 43370... Loss: 1.2819... Val Loss: 1.3657
Epoch: 10/10... Step: 43380... Loss: 1.1374... Val Loss: 1.3687
Epoch: 10/10... Step: 43390... Loss: 1.2747... Val Loss: 1.3694
Epoch: 10/10... Step: 43400... Loss: 1.1845... Val Loss: 1.3679
Epoch: 10/10... Step: 43410... Loss: 1.1329... Val Loss: 1.3686
Epoch: 10/10... Step: 43420... Loss: 1.1393... Val Loss: 1.3704
Epoch: 10/10... Step: 43430... Loss: 1.2317... Val Loss: 1.3694
Epoch: 10/10... Step: 43440... Loss: 1.2420... Val Loss: 1.3720
Epoch: 10/10... Step: 43450... Loss: 1.2316... Val Loss: 1.3735
Epoch: 10/10... Step: 43460... Loss: 1.1469... Val Loss: 1.3761
Epoch: 10/10... Step: 43470... Loss: 1.1923... Val Loss: 1.3709
Epoch: 10/10... Step: 43480... Loss: 1.1056... Val Loss: 1.3692
Epoch: 10/10... Step: 43490... Loss: 1.0615... Val Loss: 1.3689
Epoch: 10/10... Step: 43500... Loss: 1.3

Epoch: 10/10... Step: 44640... Loss: 1.2595... Val Loss: 1.3682
Epoch: 10/10... Step: 44650... Loss: 1.1903... Val Loss: 1.3705
Epoch: 10/10... Step: 44660... Loss: 1.0999... Val Loss: 1.3703


## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [107]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [108]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [109]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [110]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna. They settled in the
profit, and
shaking in his chief a study to sit divided her supple by him, and that is any which his brother was simply and down out. After at a shame was the children was the church which had
been seriously at the same part of all to him anywhere so as an institution and drove in
a
long while to surprise him, who had cauted his house as a day was in a smile, the petation was too were all disliked and stopped with his wife, though truth from
the side with a box.

"Those were its better," said that he
was coming on against the
crosing conditions.

"What am I was interested my
position."

"I've taken them."

"I am going to ball
than any of himself women," she said, and she had not had armated that his brother and
since his subject this sense of the contrary of his head and struck himself an insupper in the strange that had seen it and did not care and
take her husband, the better that
this carriages had been cruesting that had been so in the country's face there

## Loading a checkpoint

In [111]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [112]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said with a significance and most. The study, how society was silk for
mercy, at an old period of their parts, and what had trut to get about this society, had been drinking and
saw that his bridely passed tile a drinking, and the present as he had not answered in the steps of waistcoat--that had
been a life and a silence that had so said to ask him than evidently with the crowd and deceived by
the country were dull at her to comfull. He was crumed. Anything stund all that had been clearly
answered in his children, they were dressed in which he
was staying to her
boudding him. And all they were sitting and the position of his wife, they were awfuls with the burden. All
horses there arown and drank in his habling, which was all to avoid this sort of a brayers that he had been speaking
to him of any fet the candle, and that they
said she could not come. They had not have nothing, and that to strange happiness, but the summer sight to say the children's priest, which he could no